In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install transformers datasets

In [ ]:
!nvidia-smi

In [ ]:
import os
!pwd
os.chdir('/content/drive/MyDrive/Colab Notebooks/electra_for_fine_tuning')
!pwd

In [55]:
from transformers import ElectraForPreTraining, ElectraTokenizer, ElectraForMaskedLM
import torch

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-generator")

generator = ElectraForMaskedLM.from_pretrained('monologg/koelectra-base-v3-generator')
discriminator = ElectraForPreTraining.from_pretrained("monologg/koelectra-base-v3-discriminator")



# generator = torch.load('model/generator-4.pt')
# discriminator = torch.load('model/discriminator-4.pt')


loading file https://huggingface.co/monologg/koelectra-base-v3-generator/resolve/main/vocab.txt from cache at /Users/yangwoolee/.cache/huggingface/transformers/856da019f758549d6b30449d7ae8a955c7274362047cee669ffa1fcb7929f3cc.541023ff50f833a9bab3e48e78ae1856cf6744bdb336c86e797eaf675b62b2b8
loading file https://huggingface.co/monologg/koelectra-base-v3-generator/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/monologg/koelectra-base-v3-generator/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/monologg/koelectra-base-v3-generator/resolve/main/tokenizer_config.json from cache at /Users/yangwoolee/.cache/huggingface/transformers/9a0609f1cc1c5640b3b0c75cbd05668edc0957a39ec8b4f4e913ef24e3f4f989.35f013c4fd3572cfdddbbdf6223ef162dd4fb536bf83007533f201addf3287b7
loading configuration file https://huggingface.co/monologg/koelectra-base-v3-generator/resolve/main/config.json from cache at /Users/yangwoolee/.cache/huggi

In [2]:
from datasets import load_dataset

train = load_dataset('csv',data_files='data/book_train_128.csv')
validation = load_dataset('csv',data_files='data/book_validation_128.csv')

Using custom data configuration default-5b33c921347e3fef
Found cached dataset csv (/Users/yangwoolee/.cache/huggingface/datasets/csv/default-5b33c921347e3fef/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-293d316a8dadeefa
Found cached dataset csv (/Users/yangwoolee/.cache/huggingface/datasets/csv/default-293d316a8dadeefa/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
def tokenize_function(examples):
    return tokenizer(examples['sen'], max_length=128, padding=True, truncation=True)

train_data_set = train['train'].map(tokenize_function,batch_size=True)
validation_data_set = validation['train'].map(tokenize_function,batch_size=True)

Loading cached processed dataset at /Users/yangwoolee/.cache/huggingface/datasets/csv/default-5b33c921347e3fef/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-d2f2d62e57c718aa.arrow
Loading cached processed dataset at /Users/yangwoolee/.cache/huggingface/datasets/csv/default-293d316a8dadeefa/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-c7883d31a01565c7.arrow


### Domain Adaptation을 위한 Electra Model

- Electra-pytorch 라이브러리를 koElectra에 맞게 일부 수정했습니다. 

- Generator 모델은 `ElectraForMaskedLM`로, Discriminator 모델은 `ElectraForPreTraining`로 불러와야 합니다.

- 모델 학습 과정을 이해할 수 있도록 generator의 fake sentence 생성 및 Discriminator의 예측을 출력합니다.

- Electra-pytorh 원본 github 주소 : https://github.com/lucidrains/electra-pytorch



In [77]:
### Electra-pytorch 라이브러리를 KoElectra에 활용할 수 있도록 일부 변형했습니다.

### Electra로 Domain Adaptation을 수행하기 위해 개발했습니다.

### Generator 모델은 ElectraForMaskedLM로, Discriminator 모델은 ElectraForPreTraining로 불러와야 합니다.

### 더 많은 내용을 알고 싶으신 경우 Domain Adaptation Tutorial을 참고해주세요.

### Electra-pytorh 원본 github 주소 : https://github.com/lucidrains/electra-pytorch


import math
from functools import reduce
from collections import namedtuple

import torch
from torch import nn
import torch.nn.functional as F

# constants

Results = namedtuple(
    "Results",
    [
        "loss",
        "mlm_loss",
        "disc_loss",
        "gen_acc",
        "disc_acc",
        "disc_labels",
        "disc_predictions",
    ],
)


# 모델 내부에서 활용되는 함수 정의


def log(t, eps=1e-9):
    return torch.log(t + eps)


def gumbel_noise(t):
    noise = torch.zeros_like(t).uniform_(0, 1)
    return -log(-log(noise))


def gumbel_sample(t, temperature=1.0):
    return ((t / temperature) + gumbel_noise(t)).argmax(dim=-1)


def prob_mask_like(t, prob):
    return torch.zeros_like(t).float().uniform_(0, 1) < prob


def mask_with_tokens(t, token_ids):
    init_no_mask = torch.full_like(t, False, dtype=torch.bool)
    mask = reduce(lambda acc, el: acc | (t == el), token_ids, init_no_mask)
    return mask


def get_mask_subset_with_prob(mask, prob):
    batch, seq_len, device = *mask.shape, mask.device
    max_masked = math.ceil(prob * seq_len)

    num_tokens = mask.sum(dim=-1, keepdim=True)
    mask_excess = mask.cumsum(dim=-1) > (num_tokens * prob).ceil()
    mask_excess = mask_excess[:, :max_masked]

    rand = torch.rand((batch, seq_len), device=device).masked_fill(~mask, -1e9)
    _, sampled_indices = rand.topk(max_masked, dim=-1)
    sampled_indices = (sampled_indices + 1).masked_fill_(mask_excess, 0)

    new_mask = torch.zeros((batch, seq_len + 1), device=device)
    new_mask.scatter_(-1, sampled_indices, 1)
    return new_mask[:, 1:].bool()


# main electra class


class Electra(nn.Module):
    def __init__(
        self,
        generator,
        discriminator,
        tokenizer,
        *,
        num_tokens=35000,
        mask_prob=0.15,
        replace_prob=0.85,
        mask_token_id=4,
        pad_token_id=0,
        mask_ignore_token_ids=[2, 3],
        disc_weight=50.0,
        gen_weight=1.0,
        temperature=1.0,
    ):
        super().__init__()

        """
        num_tokens: 모델 vocab_size
        mask_prob: 토큰 중 [MASK] 토큰으로 대체되는 비율
        replace_prop:  토큰 중 [MASK] 토큰으로 대체되는 비율(?????)
        mask_token_i: [MASK] Token id
        pad_token_i: [PAD] Token id
        mask_ignore_token_id: [CLS],[SEP] Token id
        disc_weigh: discriminator loss의 Weight 조정을 위한 값
        gen_weigh: generator loss의 Weight 조정을 위한 값
        temperature: gumbel_distribution에 활용되는 arg, 값이 높을수록 모집단 분포와 유사한 sampling 수행
        """

        self.generator = generator
        self.discriminator = discriminator
        self.tokenizer = tokenizer

        # mlm related probabilities
        self.mask_prob = mask_prob
        self.replace_prob = replace_prob

        self.num_tokens = num_tokens

        # token ids
        self.pad_token_id = pad_token_id
        self.mask_token_id = mask_token_id
        self.mask_ignore_token_ids = set([*mask_ignore_token_ids, pad_token_id])

        # sampling temperature
        self.temperature = temperature

        # loss weights
        self.disc_weight = disc_weight
        self.gen_weight = gen_weight

    def forward(self, input_ids, **kwargs):

        try : 
            input = input_ids["input_ids"]
        except :
             input = input_ids

        # ------ 1단계 Input Data Masking --------#

        """
        - Generator는 Bert와 구조도 동일하고 학습하는 방법도 동일함. 

        - Generator 학습을 위해선 [Masked] 토큰이 필요하므로 input data를 Masking하는 과정이 필요함.

        """

        replace_prob = prob_mask_like(input, self.replace_prob)

        # do not mask [pad] tokens, or any other tokens in the tokens designated to be excluded ([cls], [sep])
        # also do not include these special tokens in the tokens chosen at random
        no_mask = mask_with_tokens(input, self.mask_ignore_token_ids)
        mask = get_mask_subset_with_prob(~no_mask, self.mask_prob)

        # get mask indices
        mask_indices = torch.nonzero(mask, as_tuple=True)

        # mask input with mask tokens with probability of `replace_prob` (keep tokens the same with probability 1 - replace_prob)
        masked_input = input.clone().detach()

        # set inverse of mask to padding tokens for labels
        gen_labels = input.masked_fill(~mask, self.pad_token_id)

        # clone the mask, for potential modification if random tokens are involved
        # not to be mistakened for the mask above, which is for all tokens, whether not replaced nor replaced with random tokens
        masking_mask = mask.clone()

        # [mask] input
        masked_input = masked_input.masked_fill(
            masking_mask * replace_prob, self.mask_token_id
        )

        # ------ 2단계 Masking 된 문장을 Generator가 학습하고 가짜 Token을 생성 --------#

        """
        - Generator를 학습하여 MLM_loss 계산(combined_loss 계산에 활용)
        - Generator에서 예측한 문장을 Discriminator 학습에 활용
        - ex) 원본 문장 : ~~~
              마스킹 문장 : 
              가짜 문장 :
        """

        # get generator output and get mlm loss(수정)
        logits = self.generator(masked_input, **kwargs).logits

        mlm_loss = F.cross_entropy(
            logits.transpose(1, 2), gen_labels, ignore_index=self.pad_token_id
        )

        # use mask from before to select logits that need sampling
        sample_logits = logits[mask_indices]

        # sample
        sampled = gumbel_sample(sample_logits, temperature=self.temperature)

        # scatter the sampled values back to the input
        disc_input = input.clone()
        disc_input[mask_indices] = sampled.detach()

        # generate discriminator labels, with replaced as True and original as False
        disc_labels = (input != disc_input).float().detach()

        # ------ 3단계 가짜 Token의 진위여부를 Discriminator가 판단하는 단계 --------#

        """
        - 가짜 문장을 학습해 개별 토큰에 대해 진위여부를 판단
        - 진짜 token이라 판단하면 0, 가짜 토큰이라 판단하면 1을 부여
        - 정답과 비교해 disc_loss를 계산(combined_loss 계산에 활용)
        - combined_loss : 학습의 최종 loss임. 모델은 combined_loss의 최솟값을 얻기 위한 방식으로 학습 진행
        """

        # get discriminator predictions of replaced / original
        non_padded_indices = torch.nonzero(input != self.pad_token_id, as_tuple=True)

        # get discriminator output and binary cross entropy loss
        disc_logits = self.discriminator(disc_input, **kwargs).logits
        disc_logits_reshape = disc_logits.reshape_as(disc_labels)

        disc_loss = F.binary_cross_entropy_with_logits(
            disc_logits_reshape[non_padded_indices], disc_labels[non_padded_indices]
        )

        # combined loss 계산
        # disc_weight을 50으로 주는 이유는 discriminator의 task가 복잡하지 않기 떄문임.
        # mlm loss의 경우 vocab_size(=35000) 만큼의 loos 계산을 수행하지만
        # disc_loss의 경우 src_token_len 만큼의 loss 계산을 수행한만큼
        # loss 값에 큰 차이가 발생함. disc_weight은 이를 보완하는 weight임.
        combined_loss = (self.gen_weight * mlm_loss + self.disc_weight * disc_loss,)

        # ------ 모델 성능 및 학습 과정을 추적하기 위한 지표(Metrics) 설계 --------#

        with torch.no_grad():
            # gen mask 예측
            gen_predictions = torch.argmax(logits, dim=-1)

            # fake token 진위 예측
            disc_predictions = torch.round(
                (torch.sign(disc_logits_reshape) + 1.0) * 0.5
            )
            # generator_accuracy
            gen_acc = (gen_labels[mask] == gen_predictions[mask]).float().mean()
            # discriminator_accuracy
            disc_acc = (
                0.5 * (disc_labels[mask] == disc_predictions[mask]).float().mean()
                + 0.5 * (disc_labels[~mask] == disc_predictions[~mask]).float().mean()
            )

        #

        return Results(
            combined_loss,
            mlm_loss,
            disc_loss,
            gen_acc,
            disc_acc,
            disc_labels,
            disc_predictions,
        )

model = Electra(generator=generator,discriminator=discriminator,tokenizer=tokenizer)
input_data = train_data_set[9999]['input_ids']

model(torch.tensor([input_data]))

Results(loss=(tensor(11.2349, grad_fn=<AddBackward0>),), mlm_loss=tensor(1.6295, grad_fn=<NllLoss2DBackward0>), disc_loss=tensor(0.1921, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), gen_acc=tensor(0.5556), disc_acc=tensor(0.8789), disc_labels=tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.]]), disc_predictions=tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.]]))

tensor([[    2,  3342,  6396,  6406,  2227,  4149,  4200,  2129,  4292,  3244,
          4219,  6243, 29365,  4181,  4129,  3178,  6522,  4283,  8328,  4192,
          2633,  4292,  3041, 24758,     3]])

In [5]:
device = 'cpu'
# optimizer = torch.optim.AdamW([{'params' : generator.parameters()},{'params' : discriminator.parameters()}], lr=2e-4, betas=(0.9, 0.999), eps=1e-09,weight_decay=0.1)
model = Electra(generator=generator,discriminator=discriminator,tokenizer=tokenizer)


In [ ]:
from transformers import TrainingArguments, TrainerCallback,Trainer,DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_eval_batch_size=8,
    per_device_train_batch_size=8,
    logging_steps=2,
    num_train_epochs=2,
    evaluation_strategy='steps'
)

class myCallback(TrainerCallback):
    """
    A bare :class:`~transformers.TrainerCallback` that just prints the logs.
    """

    def on_step_begin(self, args, state, control, logs=None, **kwargs):

        # 함수 이름.. 언제 시작할지
        # log는 설정할 때마다
        # arg,state,control은 참고할 수 있는 attribute의 경우임.
        # 근데 내가 필요한건 input
        if state.global_step % args.logging_steps == 0:
            print("")
            print(
                f"{int(state.epoch)}번째 epoch 진행 중 ------- {state.global_step}번째 step 결과"
            )


class customtrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    ############# 내용 추가
    def step_check(self):
        # state는 현 상태를 담는 attribute임.
        return self.state.global_step

    def compute_loss(self, model, inputs, return_outputs=False):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None
        prds,outputs = model(inputs)
        # Save past state if it exists
        # TODO: this needs to be fixed and made cleaner later.
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            loss = self.label_smoother(outputs, labels)
        else:
            # We don't use .loss here since the model may return tuples instead of ModelOutput.
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

        # ############# 내용 추가
        if self.step_check() % self.args.logging_steps == 0:
            print('')
            print('Discriminator 예측 정확도---*')
            print('disc_acc : ',round(outputs.disc_acc.item(),3) )
            print('----*'*20)
            print('0이 되는지 check', torch.sum(outputs.disc_predictions).item())
            print('----*'*20)
            print('loss : ',round(outputs.loss.item(),3))
            print('')

            #       #####

            # # [PAD] 제거
            # try:
            #     n = masked_input.data[0].tolist().index(3) + 1
            # except:
            #     n = None

            # # masked_sen : Masking한 문장 생성
            # masked_sen = self.tokenizer.convert_ids_to_tokens(
            #     masked_input.data[0].tolist()[:n]
            # )

            # # gen_prd : Generator 문장 예측
            # # [:n] : padding 제거
            # gen_prd = self.tokenizer.convert_ids_to_tokens(
            #     logits.unsqueeze(0).max(dim=-1)[1][0][0].tolist()[:n]
            # )

            # # gen_chg_sen : Generator 신규 문장 생성
            # gen_chg_sen = self.tokenizer.convert_ids_to_tokens(
            #     disc_input[0].tolist()[:n]
            # )

            # for i in range(len(disc_input[0][:n])):
            #     if masked_sen[i] == "[MASK]":
            #         gen_chg_sen[i] = "<< " + gen_chg_sen[i] + " >>"
        #     # step_check = 현 step 파악
        #     # args.logging_steps = argument에서 지정한 step 불러오가

        #     # batch 중 0 번째 위치한 문장 선택
        #     num = 1
        #     input_id = inputs.input_ids[num].reshape(-1).data.tolist()
        #     output_id = outputs.logits[num].argmax(dim=-1).reshape(-1).data.tolist()
        #     attention_mask = inputs.attention_mask[num]

        #     # mask가 위치한 idx 추출하기
        #     mask_idx = (inputs.input_ids[num] == 4).nonzero().data.reshape(-1).tolist()

        #     # padding 제거
        #     input_id_without_pad = [
        #         input_id[i] for i in range(len(input_id)) if attention_mask[i]
        #     ]
        #     output_id_without_pad = [
        #         output_id[i] for i in range(len(output_id)) if attention_mask[i]
        #     ]

        #     # id to token
        #     # [1:-1] [CLS,SEP] 제거
        #     inputs_tokens = self.tokenizer.convert_ids_to_tokens(input_id_without_pad)[
        #         1:-1
        #     ]
        #     outputs_tokens = self.tokenizer.convert_ids_to_tokens(
        #         output_id_without_pad
        #     )[1:-1]

        #     # output mask 부분 표시하기
        #     for i in mask_idx:
        #         # [CLS,SEP 위치 조정]
        #         outputs_tokens[i - 1] = "[" + outputs_tokens[i - 1] + "]"

        #     inputs_sen = self.tokenizer.convert_tokens_to_string(inputs_tokens)
        #     outputs_sen = self.tokenizer.convert_tokens_to_string(outputs_tokens)

        #     print(f"input 문장 : {''.join(inputs_sen)}")
        #     print(f"output 문장 : {''.join(outputs_sen)}")

        return (loss, outputs) if return_outputs else loss


trainer = customtrainer(
    model=model.to(device),
    train_dataset=train_data_set,
    eval_dataset=validation_data_set,
    args=training_args,
    tokenizer=tokenizer,
    callbacks=[myCallback],
)

trainer.train()

generator.save_pretrained('')
